In [1]:
# libs
import os
import pandas as pd
import requests
from dotenv import load_dotenv #pip3 install python-dotenv

# load secrets
load_dotenv()

True

In [2]:
# Fetch your IEX Cloud API token from the environment variable
iex_key = os.getenv('IEX_API_KEY')

# Function to fetch historical data for a stock symbol from IEX Cloud
def get_historical_data(symbol, token):
    url = f'https://cloud.iexapis.com/stable/stock/{symbol}/chart/1y?token={iex_key}'
    response = requests.get(url)
    data = response.json()
    # Convert to DataFrame
    df = pd.DataFrame(data)
    # Convert date to datetime and set as index
    df['date'] = pd.to_datetime(df['date'])
    df.set_index('date', inplace=True)
    # Ensure the data is sorted by date
    df.sort_index(inplace=True)
    return df

# Fetch historical data for Palantir (PLTR)
df_pltr = get_historical_data('PLTR', iex_key)

# Show the first few rows of the DataFrame
print(df_pltr.head())


NameError: name 'IEX_CLOUD_API_TOKEN' is not defined